# Clustering Crypto

In [41]:
# Initial imports
import pandas as pd
import hvplot.pandas
# from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [42]:
# Load the crypto_data.csv dataset.
# YOUR CODE HERE
#load the data
file_path = "C:/Users/butte/Classwork/Cryptocurrencies/crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col = 0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [43]:
# Keep all the cryptocurrencies that are being traded.
# YOUR CODE HERE
new_crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
print(new_crypto_df.shape)
new_crypto_df

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [44]:
# Keep all the cryptocurrencies that have a working algorithm.
# "Check if the Algorithm column has any null values."
# YOUR CODE HERE
for column in new_crypto_df.columns:
    print(f"Column {column} has {new_crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column IsTrading has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [45]:
# Remove the "IsTrading" column. 
# YOUR CODE HERE
new_crypto_df = new_crypto_df.drop(['IsTrading'], axis = 1)
print(new_crypto_df.shape)
new_crypto_df.head()

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [46]:
# new_crypto_df = new_crypto_df.dropna(axis = 0, how = "any")
new_crypto_df = new_crypto_df.dropna(axis = 0)
print(new_crypto_df.shape)
new_crypto_df.head()

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [47]:
# Keep the rows where coins are mined.
# YOUR CODE HERE

In [48]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE
coin_name_df = new_crypto_df.drop(['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply'], axis = 1)
print(coin_name_df.shape)
coin_name_df.head()

(685, 1)


,CoinName
42,42 Coin
404,404Coin
808,808
1337,EliteCoin
BTC,Bitcoin


In [49]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE
cluster_df = new_crypto_df.drop(['CoinName'], axis = 1)
print(cluster_df.shape)
cluster_df.head()

(685, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000


In [50]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE
cluster_encoded = pd.get_dummies(cluster_df, columns = ["Algorithm", "ProofType"])
cluster_encoded.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
808,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
# Standardize the data with StandardScaler().
# YOUR CODE HERE
cluster_scaled = StandardScaler().fit_transform(cluster_encoded)
print(cluster_scaled[0:5])

[[-0.10282804 -0.03823841 -0.03823596 -0.03823596 -0.03823596 -0.03823596
  -0.05411338 -0.07664017 -0.03823596 -0.05411338 -0.05411338 -0.03823596
  -0.03823596 -0.18216065 -0.05411338 -0.03823596 -0.03823596 -0.08574929
  -0.03823596 -0.10160947 -0.06632365 -0.03823596 -0.03823596 -0.1642757
  -0.03823596 -0.03823596 -0.13908716 -0.03823596 -0.03823596 -0.07664017
  -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.06632365 -0.03823596
  -0.07664017 -0.08574929 -0.07664017 -0.03823596 -0.03823596 -0.12775161
  -0.1335313  -0.13908716 -0.03823596 -0.05411338 -0.03823596 -0.06632365
  -0.1689039  -0.03823596 -0.03823596 -0.03823596 -0.07664017 -0.17342199
  -0.33468341 -0.03823596 -0.08574929 -0.06632365 -0.05411338 -0.03823596
   1.42042992 -0.06632365 -0.03823596 -0.03823596 -0.06632365 -0.06632365
  -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.03823596
  -0.03823596 -0.41586681 -0.03823596 -0.19054822 -0.03823596 -0.10870529
  -0.07664017 -0.09400279 -0.03823596 -

### Deliverable 2: Reducing Data Dimensions Using PCA

In [52]:
# Using PCA to reduce dimension to three principal components.
# YOUR CODE HERE
pca = PCA(n_components = 3)
crypto_pca = pca.fit_transform(cluster_scaled)

In [58]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE
df_crypto_pca = pd.DataFrame(
    data = crypto_pca, columns = ["principal component 1", "principal component 2", "principal component 3"], index = cluster_df.index
)

df_crypto_pca.head()

,principal component 1,principal component 2,principal component 3
42,-0.267147,1.464657,-0.205604
404,-0.252814,1.463056,-0.206193
808,-0.185265,0.828163,-0.114628
1337,0.251872,1.951101,-0.202006
BTC,-0.251469,-1.483555,0.052975


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [59]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE
inertia = []
k = list(range(1,11))

# loop throug K values and find the inertia
# Looking for the best k

for i in k:
    km = KMeans(n_clusters = i, random_state = 0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)
    
# create a dataframe to store the k values and their approprite inertia values.
# plot the elbow curve using hvplot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x = "k", y = "inertia", title = "Elbow Curve", xticks = k)

C:\Users\butte\anaconda3\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [60]:
# Initialize the K-Means model.
# YOUR CODE HERE
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
# YOUR CODE HERE
model.fit(df_crypto_pca)

# Predict clusters
# YOUR CODE HERE
predictions = model.predict(df_crypto_pca)

print(predictions)

[0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1
 0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0
 1 0 1 0 0 1 1 1 1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0
 1 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 0 1
 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 0 1 1 1 1
 1 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0
 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 1 0 0 0 1 1 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 1
 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0
 0 0 0 1 0 1 0 1 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 1 1 0 1 0 1 0 1 0 3 1 0
 1 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1
 1 1 0 0 0 0 0 1 0 1 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 0 1 1 1 0 0
 0 1 0 1 1 0 1 1 1 1 1 1 

In [61]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE
clustered_df = cluster_df.join(df_crypto_pca, how = "inner")
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE
clustered_df["CoinName"] = coin_name_df["CoinName"]
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE
clustered_df["Class"] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(685, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.267147,1.464657,-0.205604,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.252814,1.463056,-0.206193,404Coin,0
808,SHA-256,PoW/PoS,0.000000e+00,0,-0.185265,0.828163,-0.114628,808,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.251872,1.951101,-0.202006,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.251469,-1.483555,0.052975,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.306780,-2.132338,0.097200,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.332739,-0.847130,-0.038027,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.205940,1.380884,-0.145378,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.307273,-2.307734,0.164485,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.306703,-2.132347,0.097197,Ethereum Classic,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [66]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE
fig = px.scatter_3d(
    clustered_df,
    z="principal component 1",
    y="principal component 2",
    x="principal component 3",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
df_iris_pca.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["Class"],
    by="Class",
)